This notebook is released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).

# DIY Covid-19 Dashboard

**Student Name: Muhammad Ibad Desmukh**<br>
**Student ID: 230460524**

The following code implements a Covid-19 Dashboard which accesses data from Public Health England (PHE) which is one of many Institutions worldwide running a Coronavirus dashboard, with current statistics on the pandemic. The main endpoint to download data related to Coronavirus in the UK is https://api.coronavirus.data.gov.uk/v1/data.

**NOTE: The dashboard only shows data for periods where both 1st Dose and 2nd dose vaccination data is available.**<br>

The dashboard provides data for 1) Cumulative people vaccinated 1st dose by publish date and 2) Cumulative people vaccinated 2nd dose by publish date. It allows the user to see the trend in vaccinations, and compare the trend in vaccinations between the 1st dose and the 2nd dose, in the form of a graph.

**Cumulative people vaccinated 1st dose by publish date**<br>
Total number of people that have received a 2nd dose COVID-19 vaccination. Data are shown by the date the figures appeared in the published totals.

Daily figures include all vaccines that were given up to and including the date shown, and that were entered on the relevant system at the time of extract.

It is possible that the number of people vaccinated in surveillance figures may reduce over time, due to people dying or moving out of a resident population.

The vaccination programme began on 8 December 2020.

**Cumulative people vaccinated 2nd dose by publish date**<br>
Total number of people that have received a 2nd dose COVID-19 vaccination. Data are shown by the date the figures appeared in the published totals.

In [1]:
# import all relevant libraries and modules
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API
from ipywidgets import Layout

# customize the display features of the plot
%matplotlib inline
plt.rcParams['figure.dpi'] = 100

# open the pre-saved initial json file 
with open("vaccinations.json", "rt") as INFILE:
    jsondata=json.load(INFILE)

def wrangle_data(rawdata):
    # retrieve the list of dictionaries under the data key
    datalist=rawdata['data']

    # extract all the dates and sort them
    dates=[dictionary['date'] for dictionary in datalist]
    dates.sort()
    def parse_date(datestring):
        # Convert from date string to pandas datetime object
        return pd.to_datetime(datestring, format="%Y-%m-%d")
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])

    # define the dataframe
    index=pd.date_range(startdate, enddate, freq='D')
    df=pd.DataFrame(index=index, columns=['vaccinationsFirst', 'vaccinationsSecond'])

    # replace None values with 0
    for entry in datalist:
        date=parse_date(entry['date'])
        for column in ['vaccinationsFirst', 'vaccinationsSecond']:
            if pd.isna(df.loc[date, column]): 
                value= float(entry[column]) if entry[column]!=None else 0.0
                df.loc[date, column]=value

    # ensure that all NaN or None values are set to 0 in case any are missed
    df.fillna(0.0, inplace=True)

    # only include datapoints where data for both first dose and second dose of vaccination is available
    df = df[(df['vaccinationsFirst'] != 0) & 
        (df['vaccinationsSecond'] != 0)]
    
    return df

# assign the dataframe that is returned by wrangle_data to the variable 'df'
df=wrangle_data(jsondata)

# function to access the API
def access_api():
    # defining a filter - this is a list specifiying an areaType parameter and an optional areaName, areaCode and date parameters
    filters = [
    'areaType=nation',
    'areaName=England'
    ]

    # values here are the names of the PHE metrics
    structure = {
    "date": "date",
    "vaccinationsFirst": "cumPeopleVaccinatedFirstDoseByPublishDate",
    "vaccinationsSecond": "cumPeopleVaccinatedSecondDoseByPublishDate"
    }

    # create a Cov19API object by passing the filters and structure to its constructor
    api = Cov19API(filters=filters, structure=structure)
    # sends request to  API and retrieves the response
    jsondata=api.get_json()
    
    # write it to the json file
    with open("vaccinations.json", "wt") as OUTF:
        json.dump(jsondata, OUTF)
    
    return jsondata

# button callback function
def api_button_callback(button):
    try:
        # accesses the API and download some fresh raw data
        apidata=access_api()
        # wrangle that data into a dataframe and update the corresponding (global) variable for plotting (here, df)
        global df
        df=wrangle_data(apidata)
        apibutton.icon="check"
        apibutton.disabled=True
        # force a redraw of the graph and give the user some feedback
        refresh_graph()
    # if there is an error, give feedback to user by changing icon and description
    except:
        apibutton.icon='unlink'
        apibutton.description='Unavailable'

# create a refresh data button
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='',
    tooltip='Click to download current Public Health England data',
    icon='download'
)

apibutton.on_click(api_button_callback)

display(apibutton)

# create a label to provide messages to user
message_label = wdg.Label()

# create a multiple selection widget
vacc_cols=wdg.SelectMultiple(
    options=[
        ('Cumulative people vaccinated 1st dose by publish date', 'vaccinationsFirst'), 
        ('Cumulative people vaccinated 2nd dose by publish date', 'vaccinationsSecond')
    ],
    value=['vaccinationsFirst'],
    rows=3,
    disabled=False,
    layout=Layout(width='auto')
)

# function for generating graph
def vacc_graph(graphcolumns):
    ncols=len(graphcolumns)
    try: 
        if ncols>0:
            df.plot(kind='line', y=list(graphcolumns)) 
            plt.show()
        else:
            message_label.value='Click to select data for graph'
    # if there is any problem with the data, and it cannot be displayed show error
    except:
        message_label.value='Downloaded data could not be displayed due to an error'

# refresh the graph by automatically changing the user selection
def refresh_graph():
    vacc_graph(vacc_cols.value)
    message_label.value = 'The figure has been updated to reflect refreshed data.'

# generate graph with columns as per the multiple selection widget
output=wdg.interactive_output(vacc_graph, {'graphcolumns': vacc_cols})
display(message_label, vacc_cols, output)
vacc_cols.value

Button(description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click to download current Pu…

Label(value='')

SelectMultiple(index=(0,), layout=Layout(width='auto'), options=(('Cumulative people vaccinated 1st dose by pu…

Output()

('vaccinationsFirst',)

The graph shows a comparison of Cumulative people vaccinated 1st dose by publish date and Cumulative people vaccinated 2nd dose by publish date. Select one or both options to see 

**Author and Copyright Notice** Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england) and on the [DIY Covid Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash), Copyright (C) Fabrizio Smeraldi 2020,2023. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).